In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from keras import Sequential
from keras.layers import  Dense, Dropout
from scipy.stats import gaussian_kde
from tensorflow.python.keras import callbacks
from sklearn.preprocessing import StandardScaler
from keras.callbacks import EarlyStopping
import tensorflow as tf

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
scaler = StandardScaler()

def read_file(file_name):
	file = open(file_name)
	X = []
	k = 0
	while 1:
		line = file.readline()
		k = k+1
		if k==420000:
			break
		if not line:
			break
		data = line[:-1].split(',')
		X.append(np.array(data).astype(np.float64))
	X = np.array(X)
	return X

def process_sim_data(profile_file_path, spectral_file_path):
    # Load data from profile.h5
    with h5py.File(profile_file_path, 'r') as file:
        feature = file['feature']['block0_values'][:]

    # Load data from spectral.h5
    with h5py.File(spectral_file_path, 'r') as file:
        spectra = file['spec']['block0_values'][:]

    # Process spectra data
    for i in range(spectra.shape[0]):
        spectra[i, 0:525] = spectra[i, 0:525] / np.mean(spectra[i, 206:216])
        spectra[i, 525:1280] = spectra[i, 525:1280] / np.mean(spectra[i, 811:826])

    input_feature = np.append(feature[:, 3:7],feature[:, 9:10],axis=1)
    input_feature[:, 2:3] = np.cos(input_feature[:, 2:3])
    input_feature[:, 0:1] = np.abs(input_feature[:, 1:2] - input_feature[:, 3:4])
    input_feature = np.delete(input_feature, [1,3], axis=1)


    # Calculate xCO2 in ppm
    xco2 = feature[:, 10:11]

    xco2_ppm = xco2 * 1e6
    return spectra, input_feature, xco2_ppm

spectra_sim, feature_sim, xco2_sim = process_sim_data('profile.h5','spectral.h5')

print(spectra_sim.shape)

error_sim_file = 'error_sim.h5'
with h5py.File(error_sim_file, 'r') as file:
    error_sim = file['bad_level'][:]

for i in range(5):
    error_sim = np.append(error_sim,error_sim[:10000,:],axis=0)




In [ ]:
years = np.zeros([60000,1])
years[:10000] = 2016
years[10000:20000] = 2017
years[20000:30000] = 2018
years[30000:40000] = 2019
years[40000:50000] = 2020
years[50000:60000] = 2021

In [ ]:
def process_exp_file(file_path,p_file):

    
    with h5py.File(file_path, 'r') as h5file:
        x = h5file['x'][:]
        error = h5file['bad_level'][:]
        # y = h5file['y'][:]
    print(x.shape)
    sample_size = x.shape[0]

    y = np.asarray(read_file(p_file))

    feature = np.append(y[:,4:8],y[:,14:15],axis=1)
    feature[:,2:3] = np.cos(feature[:,2:3])
    feature[:,0:1] = np.abs(feature[:,1:2] - feature[:,3:4])
    feature = np.delete(feature, [1,3], axis=1)

    input_nn = np.append(x, feature, axis=1)
    ouput_nn = y[:, 11:12]
    error_nn = error[:,:]
    
    # # add year
    year = int(file_path.split('_')[-1].split('.')[0]) + 2000
    input_nn = np.append(input_nn, np.full((sample_size, 1), year), axis=1)
    
    return sample_size, input_nn, ouput_nn, error_nn


file_paths = [
    'exp_16.h5',
    'exp_17.h5',
    'exp_18.h5',
    'exp_19.h5',
    'exp_20.h5',
    'expplume_17.h5',
    'expplume_20.h5'
]

p_paths = [
    'train_16.dat',
    'train_17.dat',
    'train_18.dat',
    'train_19.dat',
    'train_20.dat',
    'trainplume_17.dat',
    'trainplume_20.dat'
]

input_nn_test = []
xco2_test = []
num_list = []
error_test = []
for file_path,p_path in zip(file_paths,p_paths):
    sample_data_num, sampled_data_in, sampled_data_xco2, sampled_error = process_exp_file(file_path,p_path)
    input_nn_test.append(sampled_data_in)
    xco2_test.append(sampled_data_xco2)
    num_list.append(sample_data_num)
    error_test.append(sampled_error)

input_nn_test = np.vstack(input_nn_test)
error_test = np.vstack(error_test)
xco2_test = np.vstack(xco2_test)
num_pro = np.array(num_list)

print("Input features shape:", input_nn_test.shape)
print("xco2 shape:", xco2_test.shape)
print("shape:", num_pro.shape)

In [ ]:
error_16_18 = np.append(error_test[:38626,:],error_test[100421:102421,:],axis=0)
error_16_18.shape

In [ ]:
union_max = (error_16_18 >0).any(axis=0).astype(float)

indices_to_drop = np.where(union_max > 0.5)[0]

input_nn_test = np.delete(input_nn_test, indices_to_drop, axis=1)
spectra_sim = np.delete(spectra_sim, indices_to_drop, axis=1)

In [ ]:

model_label = '2band_2angle_year_retp_error_xco2'

input_nn = np.append(spectra_sim,feature_sim,axis=1)#spectra
input_nn = np.append(input_nn,years,axis=1)


scaler.fit(input_nn)  
[ux1,sx1]=[scaler.mean_,scaler.var_]
input_nn_test = scaler.transform(input_nn_test)
input_nn = scaler.transform(input_nn)


scaler.fit(xco2_sim)  
[uy1,sy1]=[scaler.mean_,scaler.var_]
output_nn_test = scaler.transform(xco2_test)
output_nn = scaler.transform(xco2_sim)



input_nn = np.append(input_nn,input_nn_test[0:38626,:],axis=0)#combined 2016 samples
output_nn = np.append(output_nn,output_nn_test[0:38626,:],axis=0)

np.savetxt("ux_"+model_label+".dat",ux1, fmt="%15.5e",delimiter=',')
np.savetxt("sx_"+model_label+".dat",sx1, fmt="%15.5e",delimiter=',')
np.savetxt("uy_"+model_label+".dat",uy1, fmt="%15.5e",delimiter=',')
np.savetxt("sy_"+model_label+".dat",sy1, fmt="%15.5e",delimiter=',')



In [ ]:
train_X, test_X, train_y, test_y = train_test_split(input_nn,output_nn,test_size=0.1, random_state=0)
print(input_nn.shape,output_nn.shape)

from tensorflow.keras.losses import Huber
huber_loss = Huber(delta=1.0)


In [ ]:
layer=[1000,500,300,100,20]
n_features = input_nn.shape[1]
filepath="nn/train_"+model_label+".h5"

model = Sequential()

model.add(Dense(layer[0], activation='relu', kernel_initializer='he_uniform', input_shape=(n_features,)))
model.add(Dropout(0.1))
model.add(Dense(layer[1], activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(layer[2], activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(layer[3], activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(layer[4], activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(output_nn.shape[1]))



model.compile(optimizer='adam', loss=huber_loss,metrics=['mae'])
model.summary()
checkpoint=callbacks.ModelCheckpoint(filepath,monitor='val_loss',verbose=0,save_best_only=True,mode='min')

early_stopping = EarlyStopping(
    monitor='val_loss',   
    patience=15,          
    mode='min',           
    verbose=1             
)
callbacks_list=[checkpoint,early_stopping]


model.fit(train_X, train_y, epochs=1000, batch_size=128,callbacks=callbacks_list, validation_split = 0.1  ,verbose=1)


In [ ]:
# (38626, 1280)
# (39850, 1280) 78476
# (35945, 1280) 114421
# (36452, 1280) 150873
# (43277, 1280) 194150
res = model.predict(test_X)
MLP = scaler.inverse_transform(res)
LBL = scaler.inverse_transform(test_y)


def r2(y_true, y_pred):
    """
    Calculation of the unadjusted r-squared, goodness of fit metric
    """
    sse  = np.square( y_pred - y_true ).sum()
    sst  = np.square( y_true - y_true.mean() ).sum()
    return 1 - sse/sst

xy = (np.append(MLP, LBL,axis=1)).T
z = gaussian_kde(xy)(xy)
z=(z-np.min(z))/(np.max(z)-np.min(z))
idx = z.argsort()
MMLP, LLBL, z = MLP[idx], LBL[idx], z[idx]
slope, intercept = np.polyfit(LLBL[:,0], MMLP[:,0], 1)
xyline = np.linspace(0.95*np.min(LBL),1.05*np.max(LBL),61)

fig = plt.figure(figsize=(8,7))
figg,ax = plt.subplots()
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.ylabel('Predicted [ppm]',fontname="serif",fontsize=14)
plt.xlabel('Ideal/OCO-2 [ppm]',fontname="serif",fontsize=14)
plt.axis([390,425,390,425])
text1 = 'N:'+str(len(LBL)) 
text2 = 'R$^2$:'+str("%.3f" % r2(LBL,MLP))
text3 = 'ME: '+str("%.3f" % np.mean(LBL-MLP))+' ppm'
text4 = 'RMSE: '+str("%.3f" % np.sqrt(mean_squared_error(LBL, MLP)))+' ppm'
plt.plot(xyline, xyline, 'r-',label='Ideal$\pm$1%', linewidth=2)
plt.fill_between(xyline, xyline*1.01, xyline*0.99,
    alpha=0.5, edgecolor='0.4', facecolor='0.4',
    linewidth=1, linestyle='--', antialiased=True)
plt.scatter(LLBL, MMLP, c=z,s=7, alpha=1.0)#facecolors='none',
plt.colorbar(label='Number density')
plt.tick_params(labelsize=14)
plt.legend(fontsize=14)
plt.ticklabel_format(style='plain', scilimits=(0, 0), axis='both')
plt.text(0.02,0.85,text1,ha='left',va='top',transform=ax.transAxes,fontname="serif",fontsize=14)
plt.text(0.02,0.80,text2,ha='left',va='top',transform=ax.transAxes,fontname="serif",fontsize=14)
plt.text(0.02,0.75,text3,ha='left',va='top',transform=ax.transAxes,fontname="serif",fontsize=14)
plt.text(0.02,0.70,text4,ha='left',va='top',transform=ax.transAxes,fontname="serif",fontsize=14)
plt.tight_layout()
plt.savefig(model_label+"_testdata.png",dpi=300)